Try chatgpt for your task before fine-tuning. If chatgpt is not enough, then fine-tune it. Do not forget that chat-gpt may be sufficient for most simple tasks.

To fine-tune the GPT-3.5 model, you need to convert each observation in your data to the following message template (Jsonline Format):

In [ ]:
# """{'messages': [{'role': 'system', 'content': 'Given a text, provide the following fields in a JSON format, where applicable:\
#                                              'person (only name/surname)', 'time (week, month, summer etc.)', 'currency (USD, EURO etc.)',\
#                                               and 'place (country, city etc.)'.'},
#                  {'role': 'user', 'content': 'The White House is located in Washington, D.C.'},
#                  {'role': 'assistant', 'content': "{'Person': 'null', 'Time': 'null', 'Currency': 'null', 'Place': 'Washington, D.C.'}"}]}"""

# system: the instructions you give to the model.
# user: the feature/independent variable/text of each observation in your data (X)
# assistant: the dependent variable/target/label of each observation in your data (y).

In [1]:
!pip install -qU openai

In [2]:
!pip install -q tiktoken

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [4]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY']=userdata.get('OPENAI_API_KEY')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
from openai import OpenAI

client = OpenAI() #api_key=os.environ['OPENAI_API_KEY']

We're getting 20 observations ready for NER fine tuning.

In [ ]:
prompt_train= ["After spending several months studying abroad in Spain, John Wick returned home with a newfound appreciation for the Spanish language and culture.",
         "Despite the fact that I had saved up 100.000 USD, I still found it difficult to afford a house in the city.",
         "When I was a child, my grandparents would take me on long walks through the countryside, and those memories are still some of my most cherished.",
         "After years of working in the corporate world, I decided to start my own business, and I've never looked back.",
         "The ancient ruins we visited in Rome were so awe-inspiring that I found myself speechless.",
         "Despite the fact that I had been practicing for months, my piano recital was a disaster, and I was humiliated.",
         "When she traveled to Thailand, Mary was amazed by the beauty and serenity of the Buddhist temples.",
         "My grandmother's estate, which had been passed down through several generations of our family, was the site of many cherished family gatherings.",
         "When I was a student, I spent a semester studying abroad in France, and it was one of the most enriching experiences of my life.",
         "After my grandfather passed away, my family and I spent weeks going through his belongings and reminiscing about his life.",
         "The White House is located in Washington, D.C.",
         "I need to exchange dollars for euros before I travel to Europe.",
         "The exchange rate between the pound and the euro is favorable.",
         "As a child, I used to spend my summers at my grandparents' farm, and those days are some of my most cherished memories.",
         "After years of working for a multinational corporation, I decided to quit my job and travel the World.",
         "My great-grandfather fought in World War II, and his stories about the war have been passed down through the generations of our family.",
         "When I was in college, I spent a semester studying abroad in China, and it was an eye-opening experience.",
         "My best friend and I have been planning a trip to South America for years, and we're finally going to make it happen this summer.",
         "My parents own a small motel in a charming seaside town, and they've been running it for over 20 years.",
         "Johnson and I decided to take a road trip across the United States during the summer after we graduated from college."]

In [ ]:
# Çıktıyı istediğiniz gibi düzenleyebilirsiniz. Çıktının sözlük formatında olmasını istedik, bu yüzden onu bu şekilde formatladık.

target_train= [{"Person": "John Wick", "Time": "several months", "Currency": "null", "Place": "Spain"},
         {"Person": "null", "Time": "null", "Currency": "USD", "Place": "null"},
         {"Person": "null", "Time": "null", "Currency": "null", "Place": "null"},
         {"Person": "null", "Time": "years", "Currency": "null", "Place": "null"},
         {"Person": "null", "Time": "null", "Currency": "null", "Place": "Rome"},
         {"Person": "null", "Time": "months", "Currency": "null", "Place": "null"},
         {"Person": "Mary", "Time": "null", "Currency": "null", "Place": "Thailand"},
         {"Person": "null", "Time": "null", "Currency": "null", "Place": "null"},
         {"Person": "null", "Time": "semester", "Currency": "null", "Place": "France"},
         {"Person": "null", "Time": "weeks", "Currency": "null", "Place": "null"},
         {"Person": "null", "Time": "null", "Currency": "null", "Place": "Washington, D.C."},
         {"Person": "null", "Time": "null", "Currency": ["dollars", "euros"], "Place": "Europe"},
         {"Person": "null", "Time": "null", "Currency": ["pound", "euro"], "Place": "null"},
         {"Person": "null", "Time": ["summers", "those days"], "Currency": "null", "Place": "null"},
         {"Person": "null", "Time": "years", "Currency": "null", "Place": "World"},
         {"Person": "null", "Time": "null", "Currency": "null", "Place": "null"},
         {"Person": "null", "Time": "semester", "Currency": "null", "Place": "China"},
         {"Person": "null", "Time": ["years", "summer"], "Currency": "null", "Place": "South America"},
         {"Person": "null", "Time": "over 20 years", "Currency": "null", "Place": "null"},
         {"Person": "Johnson", "Time": "summer", "Currency": "null", "Place": "United States"}]

In [ ]:
prompt_test=["My grandmother always talks about the time she spent in Paris when she was young, and how it was the most magical experience of her life.",
      "After receiving his bonus in USD, Mark decided to invest in the stock market.",
      "My cousin Sarah plans to visit Tokyo next spring.",
      "During the winter holidays, we traveled to the Swiss Alps to enjoy skiing and the beautiful scenery.",
      "Jennifer won a cash prize of 1000 euros at the competition in Berlin."
]

In [ ]:
target_test= [{"Person": "null", "Time": "null", "Currency": "null", "Place": "Paris"},
              {"Person": "Mark", "Time": "null", "Currency": "USD", "Place": "null"},
              {"Person": "Sarah", "Time": "next spring", "Currency": "null", "Place": "Tokyo"},
              {"Person": "null", "Time": "winter holidays", "Currency": "null", "Place": "Swiss Alps"},
              {"Person": "Jennifer", "Time": "null", "Currency": "euros", "Place": "Berlin"}
]

In [ ]:
my_dict_train={"text":prompt_train, "target":target_train}
train_df=pd.DataFrame(my_dict_train)
train_df

,text,target
0,After spending several months studying abroad ...,"{'Person': 'John Wick', 'Time': 'several month..."
1,Despite the fact that I had saved up 100.000 U...,"{'Person': 'null', 'Time': 'null', 'Currency':..."
2,"When I was a child, my grandparents would take...","{'Person': 'null', 'Time': 'null', 'Currency':..."
3,"After years of working in the corporate world,...","{'Person': 'null', 'Time': 'years', 'Currency'..."
4,The ancient ruins we visited in Rome were so a...,"{'Person': 'null', 'Time': 'null', 'Currency':..."
5,Despite the fact that I had been practicing fo...,"{'Person': 'null', 'Time': 'months', 'Currency..."
6,"When she traveled to Thailand, Mary was amazed...","{'Person': 'Mary', 'Time': 'null', 'Currency':..."
7,"My grandmother's estate, which had been passed...","{'Person': 'null', 'Time': 'null', 'Currency':..."
8,"When I was a student, I spent a semester study...","{'Person': 'null', 'Time': 'semester', 'Curren..."
9,"After my grandfather passed away, my family an...","{'Person': 'null', 'Time': 'weeks', 'Currency'..."


In [ ]:
my_dict_test={"text":prompt_test, "target":target_test}
test_df=pd.DataFrame(my_dict_test)
test_df

,text,target
0,My grandmother always talks about the time she...,"{'Person': 'null', 'Time': 'null', 'Currency':..."
1,"After receiving his bonus in USD, Mark decided...","{'Person': 'Mark', 'Time': 'null', 'Currency':..."
2,My cousin Sarah plans to visit Tokyo next spring.,"{'Person': 'Sarah', 'Time': 'next spring', 'Cu..."
3,"During the winter holidays, we traveled to the...","{'Person': 'null', 'Time': 'winter holidays', ..."
4,Jennifer won a cash prize of 1000 euros at the...,"{'Person': 'Jennifer', 'Time': 'null', 'Curren..."


In [ ]:
X_train=train_df["text"]
y_train=train_df["target"]

X_test=test_df["text"]
y_test=test_df["target"]

In [ ]:
system="""Extract the labels delimited by XML tags from text delimited by XML tags and return them in dict type.
<labels>'Person': <person names and surnames>, 'Time': <word or words indicating time \
like this week, last month, summer, next month etc.>', 'Currency':<all currencies like USD, EURO, etc.>, \
'Place':<settlement like country, city, district etc.> </labels>"""
system
# commonly used delimiters:
# triple quotes: """
# triple backticks: '''
# riple dashes: ___
# angle brackets : <>
# curly brackets: {}
# XML tags: <tag> </tag> ---> <text>This is sentence</text>

"Extract the labels delimited by XML tags from text delimited by XML tags and return them in dict type.\n<labels>'Person': <person names and surnames>, 'Time': <word or words indicating time like this week, last month, summer, next month etc.>', 'Currency':<all currencies like USD, EURO, etc.>, 'Place':<settlement like country, city, district etc.> </labels>"

In [ ]:
#system="""Extract {labels} from text and return them in dict type.
#labels: 'Person (name, surname)', 'Time (this week, last month, summer, next month etc.)', 'Currency (USD, EURO, etc.)', 'Place (country, city, etc.)'.""" #

In [ ]:
import json
import pandas as pd

def convert_df_to_dict(X, y):
  """Converts X and y to dictionary format.

  Args:
    X: text
    y: label

  Returns:
    A dataframe(as X and y) in dictionary format.
  """
  #df = pd.read_csv(csv_file)
  dict_data = []
  for i, j in zip(X, y):
    dict_data.append({
        "messages": [
            {
                "role": "system",
                "content": system
            },
            {
                "role": "user",
                "content": '<text>'+str(i)+'</text>'
            },
            {
                "role": "assistant",
                "content": str(j) # Dictionary formatındaki assistant content'i string'e dönüştürüyoruz.
            }                     # Tüm contentler string'lerden oluşmalıdır, aksi takdirde Format validation bölümünde hata alırsınız.
        ]
    })

  return dict_data

In [ ]:
train=convert_df_to_dict(X_train, y_train)
test=convert_df_to_dict(X_test, y_test)

In [ ]:
train[0]

{'messages': [{'role': 'system',
   'content': "Extract the labels delimited by XML tags from text delimited by XML tags and return them in dict type.\n<labels>'Person': <person names and surnames>, 'Time': <word or words indicating time like this week, last month, summer, next month etc.>', 'Currency':<all currencies like USD, EURO, etc.>, 'Place':<settlement like country, city, district etc.> </labels>"},
  {'role': 'user',
   'content': '<text>After spending several months studying abroad in Spain, John Wick returned home with a newfound appreciation for the Spanish language and culture.</text>'},
  {'role': 'assistant',
   'content': "{'Person': 'John Wick', 'Time': 'several months', 'Currency': 'null', 'Place': 'Spain'}"}]}

In [ ]:
import json

with open("train.jsonl", "w") as f:
    for item in train:
        f.write(json.dumps(item) + "\n") # Python'da bir dictionary'i JSON formatına dönüştürmek için json.dumps() fonksiyonu kullanılır.
                                         # Bu fonksiyon dictionary'i bir JSON string'e dönüştürür.

# İnce ayar için modele besleyeceğimiz train ve test datası JSON line formatında olmalıdır. Bunun için dictionary formatındaki tüm gözlemleri
# JSON line formatına dönüştürüyoruz.

# JSON formatı dictionary formatının string'e dönüştürülmesidir.

# dict format: {'name': 'johnson', 'age':44}
# json format: "{'name': 'johnson', 'age':'44'}"

In [ ]:
with open("test.jsonl", "w") as f:
    for item in test:
        f.write(json.dumps(item) + "\n")

## Data preparation and analysis for chat model fine-tuning

**Data loading**, **Format validation**, **Token Counting Utilities**, **Data Warnings and Token Counts**, ve **Cost Estimation** bölümleri için aşağıdaki kod blokları OpenAI tarafından hazırlanmıştır. Ve ince ayardan önce kullanılması tavsiye edilir.

Bu kod blokları, bir chatgpt modeline ince ayar yapmak için kullanılan datayı önceden işlemek ve analiz etmek için bir araç görevi görür. Biçim hatalarını kontrol eder, temel istatistikler sağlar ve ince ayar maliyetini belirlemek için gerekli olan token sayılarını tahmin eder.

In [ ]:
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

## Data loading

We first load the chat dataset from an example JSONL file.

In [ ]:
data_path = "/content/train.jsonl" # data_path değişkeni jsonl dosyasının yolunu belirtir.

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f: # with open() dosyayı açar ve json.loads() işlevini
                                                  # kullanarak her satırı bir dict nesnesi olarak yükler.

    dataset = [json.loads(line) for line in f] # tüm dict nesnelerini dataset listesine atıyoruz.

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 20
First example:
{'role': 'system', 'content': "Extract the labels delimited by XML tags from text delimited by XML tags and return them in dict type.\n<labels>'Person': <person names and surnames>, 'Time': <word or words indicating time like this week, last month, summer, next month etc.>', 'Currency':<all currencies like USD, EURO, etc.>, 'Place':<settlement like country, city, district etc.> </labels>"}
{'role': 'user', 'content': '<text>After spending several months studying abroad in Spain, John Wick returned home with a newfound appreciation for the Spanish language and culture.</text>'}
{'role': 'assistant', 'content': "{'Person': 'John Wick', 'Time': 'several months', 'Currency': 'null', 'Place': 'Spain'}"}


## Format validation

Veri kümesindeki her bir gözlemin ince ayar API'sinin beklediği formata uygun olduğunu doğrulamak için çeşitli hata kontrolleri gerçekleştirebiliriz. Hatalar, daha kolay hata ayıklama için niteliklerine göre kategorize edilir.

1.**Data Type Check(Veri Tipi Kontrolü)**:Veri setindeki her girdinin(input) bir dict olup olmadığını kontrol eder. Error type: data_type.

2.**Presence of Message List(Mesaj Listesinin Varlığı)**: Her girdi için mesaj listesinin olup olmadığını kontrol eder. Error type: missing_messages_list.

3.**Message Keys Check(Mesaj Anahtarları Kontrolü)**: Mesajların içerisindeki "role" ve "content" anahtarlarının varlığını kontrol eder. Error type: message_missing_key.

4.**Unrecognized Keys in Messages(Mesajlardaki Tanınmayan Anahtarlar)**: Bir mesajın "role", "content" ve role_name(system, user, assistant) dışında anahtarları olup olmadığını kontrol eder. Error type: message_unrecognized_key.

5.**Role Validation(role doğrulaması)**: "Role"lerin "system", "user", veya "assistant" olduğunu teyit eder. Error type: unrecognized_role.

6.**Content Validation(content doğrulaması)**: contentin string bir ifade olduğunu teyit eder. Error type: missing_content.

7.**Assistant Message Presence(Asistan Mesajı Varlığı:)**: mesajın içinde assistant'a ait bir içerik olup olmadığını kontrol eder. Error type: example_missing_assistant_message.

Aşağıdaki kod, bu kontrolleri gerçekleştirir ve bulunan her türlü hatanın sayısını yazdırır. Bu, hata ayıklama ve veri kümesinin bir sonraki adımlara hazır olup olmadığını kontrol etme açısından faydalıdır.

In [ ]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        if not content or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

# bu fonksiyon, verilerin ince ayar için hazır olup olmadığını kontrol eder. Eksik veya geçersiz veriler için hata döndürür.
# datada problem yoksa "Hata bulunamadı" mesajını döndürür.

No errors found


## Token Counting Utilities

Lets define a few helpful utilities to be used in the rest of the notebook.

In [ ]:
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.05)}, {np.quantile(values, 0.95)}")

## Data Warnings and Token Counts

Aşağıdaki analizlerle, verilerdeki potansiyel sorunları, eksik mesajları belirleyebilir ve mesaj ve token sayıları hakkında istatistiksel bilgiler sağlayabiliriz.

1.**Missing System/User Messages**: "system" veya "user" mesajı eksik olan gözlemleri sayar. Bu mesajlar, assistant'ın davranışını tanımlamak ve konuşmayı başlatmak için kritik öneme sahiptir.

2.**Number of Messages Per Example**: Her bir gözlemdeki mesaj (system, user, assistant) sayısının dağılımını özetler ve diyalog karmaşıklığı hakkında bilgi sağlar. Her mesajın 3 ayrı bölümü olup olmadığını kontrol eder: system, user, assistant. Öyleyse, her mesajdaki parça sayısının min, max, mean, median, %5 ve %95 dilimlerdeki değeri her zaman 3 olacaktır. Aksi takdirde, bu değerler 3'ten farklı olacaktır.

3.**Total Tokens Per Example**: Her gözlemdeki toplam token sayısının dağılımını hesaplar ve özetler. İnce ayar maliyetlerini anlamak için önemlidir. Bu, system, user ve assistant parçaları da dahil olmak üzere her mesajdaki toplam token sayısı ve dağılımları (mean, median, minimum, maximum, etc.) hakkında bilgi sağlar.

4.**Tokens in Assistant's Messages**: her gözlemdeki assistant mesajlarındaki toekn sayısını hesaplar ve bu dağılımı özetler (mean, median, minimum, maximum, etc.). Assistant hakkında bilgi sağlar.

5.**Token Limit Warnings**:  Herhangi bir gözlem maksimum toekn sınırını (65536 token) aşarsa kontrol eder, çünkü bu tür gözlemler ince ayar sırasında kırpılacak ve bu da veri kaybına neden olacaktır.

In [ ]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 65536 for l in convo_lens) #gpt-3.5 için 16385
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 146, 164
mean / median: 156.05, 156.5
p5 / p95: 147.9, 164.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 24, 29
mean / median: 25.6, 24.5
p5 / p95: 24.0, 29.0

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning


## Cost Estimation

Bu son bölümde, ince ayar için kullanılacak toplam token sayısını tahmin ediyoruz, bu da maliyeti yaklaşık olarak tahmin etmemizi sağlıyor. Token sayısı arttıkça ince ayar işlemlerinin süresinin de artacağını belirtmekte fayda var.

In [ ]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 65536

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

current_price=3 # bir milyon token başına fiyat, lütfen güncel fiyat için openai web sayfasını kontrol edin.
print(f"Estimate total costs ~{(n_epochs * n_billing_tokens_in_dataset)/1000000 * current_price}")

# İnce ayar için modele en az 10 gözlem verilmelidir. Aksi takdirde model hata döndürecektir. Ancak ince ayar için modele minimum 100, maksimum 25000
# gözlem verilmesi tavsiye edilir.

# GPT modelleri, ince ayar için varsayılan olarak 3 epoch ile çalışır. Ancak bu kod, verilerinizdeki gözlem sayısına göre minimum 1'den maksimum 25'e
# kadar kaç epoch eğitim yapmanız gerektiğini size önerir.

# Tüm eğitim boyunca işlenecek token sayısını ve bu token sayısına göre ince ayar maliyetini döndürür.

# NOT: Bu bölüm yalnızca eğitim verilerine uygulanacaktır.

Dataset has ~3121 tokens that will be charged for during training
By default, you'll train for 5 epochs on this dataset
By default, you'll be charged for ~15605 tokens
Estimate total costs ~0.046815


In [ ]:
training_response = client.files.create(
                                        file=open("/content/train.jsonl", "rb"),
                                        purpose="fine-tune") # Train datasını ince ayar için hazırlıyoruz.

training_file_id = training_response.id # Hazır olan train datası için oluşturulan ID’yi çekiyoruz.


validation_response = client.files.create(
                                        file=open("/content/test.jsonl", "rb"),
                                        purpose="fine-tune") # Test datasını ince ayar için hazırlıyoruz.

validation_file_id = validation_response.id

print("Training file id:", training_file_id)
print("Validation file id:", validation_file_id)

Training file id: file-uyhpnihtjozF9JK0y34ZQUK1
Validation file id: file-Z1RarPDddirYwvf3B5v7ffCC


In [ ]:
suffix_name = "ner 4"

response = client.fine_tuning.jobs.create(
    training_file=training_file_id, # Train datasının ID'si ince ayar modeline veriliyor. Train datasının ID'si mutlaka sağlanmalıdır
    validation_file=validation_file_id, #Validation datsının ID'si ince ayar modeline veriliyor. Bu kimliği sağlamak isteğe bağlıdır
    model="gpt-4o-mini-2024-07-18", # "gpt-3.5-turbo"
    suffix=suffix_name, # İnce ayar yapılan modelin ismine kendi seçeceğimiz bir ek ekleyebiliyoruz ancak modelin ismini tam olarak belirleyemiyoruz.
    hyperparameters={"n_epochs":5}, # Yalnızca Epoch, batch size parametresini ayarlamamıza izin veriliyor. Diğer parametreleri düzenlememize izin verilmiyor.
                                    # Önerilen epoch sayısı 3 olduğundan epoch'u 3 olarak ayarladık.
)

job_id = response.id # The ID of fine-tune model.

print(response)

FineTuningJob(id='ftjob-aCzu49O9poTR2wJBdRNUZjsg', created_at=1723361413, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=5, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-b88r78OSLP9wnPnDMsswFKhJ', result_files=[], seed=998221641, status='validating_files', trained_tokens=None, training_file='file-uyhpnihtjozF9JK0y34ZQUK1', validation_file='file-Z1RarPDddirYwvf3B5v7ffCC', estimated_finish=None, integrations=[], user_provided_suffix='ner 4')


In [ ]:
job_id=client.fine_tuning.jobs.list().data[0].id
job_id

'ftjob-aCzu49O9poTR2wJBdRNUZjsg'

In [ ]:
job_id

'ftjob-aCzu49O9poTR2wJBdRNUZjsg'

In [ ]:
#client.fine_tuning.jobs.retrieve(job_id) # İnce ayar hakkında genel bilgiler alıyoruz.

FineTuningJob(id='ftjob-aCzu49O9poTR2wJBdRNUZjsg', created_at=1723361413, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=5, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-b88r78OSLP9wnPnDMsswFKhJ', result_files=[], seed=998221641, status='validating_files', trained_tokens=None, training_file='file-uyhpnihtjozF9JK0y34ZQUK1', validation_file='file-Z1RarPDddirYwvf3B5v7ffCC', estimated_finish=None, integrations=[], user_provided_suffix='ner 4')

In [ ]:
#client.fine_tuning.jobs.retrieve(job_id)

FineTuningJob(id='ftjob-aCzu49O9poTR2wJBdRNUZjsg', created_at=1723361413, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=5, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-b88r78OSLP9wnPnDMsswFKhJ', result_files=[], seed=998221641, status='running', trained_tokens=None, training_file='file-uyhpnihtjozF9JK0y34ZQUK1', validation_file='file-Z1RarPDddirYwvf3B5v7ffCC', estimated_finish=None, integrations=[], user_provided_suffix='ner 4')

In [ ]:
client.fine_tuning.jobs.retrieve(job_id)

FineTuningJob(id='ftjob-aCzu49O9poTR2wJBdRNUZjsg', created_at=1723361413, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal:ner-4:9uxSYzGm', finished_at=1723361956, hyperparameters=Hyperparameters(n_epochs=5, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-b88r78OSLP9wnPnDMsswFKhJ', result_files=['file-lxucRnZDvrP7NBhFt39M9qkK'], seed=998221641, status='succeeded', trained_tokens=15465, training_file='file-uyhpnihtjozF9JK0y34ZQUK1', validation_file='file-Z1RarPDddirYwvf3B5v7ffCC', estimated_finish=None, integrations=[], user_provided_suffix='ner 4')

In [ ]:
client.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id, limit=10) # Son 10 adımdaki ince ayar işleminin olaylarını görüntüler.

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-eRB1CQoSLWz9aRFj7yHav69s', created_at=1723361966, level='info', message='The job has successfully completed', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-Q28kYeQDo5NFfAQjjmHxMBMw', created_at=1723361959, level='info', message='New fine-tuned model created: ft:gpt-4o-mini-2024-07-18:personal:ner-4:9uxSYzGm', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-mZx1F2DYMlKE1oZximftuqdA', created_at=1723361959, level='info', message='Checkpoint created at step 80 with Snapshot ID: ft:gpt-4o-mini-2024-07-18:personal:ner-4:9uxSYMyw:ckpt-step-80', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-z2qTxNM2urwfEUQYtBqusODk', created_at=1723361959, level='info', message='Checkpoint created at step 60 with Snapshot ID: ft:gpt-4o-mini-2024-07-18:personal:ner-4:9uxSY9yn:ckpt-step-60', object='fine_tuning.job.

In [ ]:
response= client.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id, limit=100)

events=response.data

events.reverse() # İnce ayar işleminin ilk adımları başlangıçta olacak şekilde, rever() fonksiyonunu kullanarak
                 # ince ayar adımlarının sırasını tersine çeviriyoruz.

for event in events:
  print(event.message)

# Her adım için training loss skorlarını alıyoruz.

Step 5/100: training loss=0.73, validation loss=0.54
Step 6/100: training loss=0.57, validation loss=1.61
Step 7/100: training loss=0.67, validation loss=0.59
Step 8/100: training loss=0.61, validation loss=0.39
Step 9/100: training loss=0.34, validation loss=0.11
Step 10/100: training loss=0.63, validation loss=0.17
Step 11/100: training loss=0.22, validation loss=0.19
Step 12/100: training loss=0.61, validation loss=0.06
Step 13/100: training loss=0.08, validation loss=0.06
Step 14/100: training loss=0.03, validation loss=0.15
Step 15/100: training loss=0.42, validation loss=0.10
Step 16/100: training loss=0.10, validation loss=0.06
Step 17/100: training loss=0.03, validation loss=0.01
Step 18/100: training loss=0.10, validation loss=0.05
Step 19/100: training loss=0.01, validation loss=0.07
Step 20/100: training loss=0.17, validation loss=0.03, full validation loss=0.01
Step 21/100: training loss=0.01, validation loss=0.00
Step 22/100: training loss=0.00, validation loss=0.01
Step 2

In [ ]:
#client.fine_tuning.jobs.cancel(job_id)

# You can cancel the fine-tuning process you have started with this code. However, the fine-tuning process will start in 5-10 minutes,
# so you need to cancel it before that. If you don't, you won't be able to cancel it.

FineTuningJob(id='ftjob-nKraJjw1ggsXvBT8TNkrSoZw', created_at=1713810376, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=5, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-b88r78OSLP9wnPnDMsswFKhJ', result_files=[], seed=483893693, status='cancelled', trained_tokens=None, training_file='file-QETofvgMzM3vaF3XZ7HywbLP', validation_file=None, integrations=[], user_provided_suffix='NER')

In [ ]:
import numpy as np
np.exp(-0.07)

0.9323938199059483

In [ ]:
np.log(0.9323938199059483)

-0.06999999999999995

## Prediction

In [ ]:
client.fine_tuning.jobs.retrieve(job_id) # İnce ayar hakkında genel bilgiler alıyoruz öncelikle. burdan fine-tune model ismini çekeceğiz

FineTuningJob(id='ftjob-aCzu49O9poTR2wJBdRNUZjsg', created_at=1723361413, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal:ner-4:9uxSYzGm', finished_at=1723361956, hyperparameters=Hyperparameters(n_epochs=5, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-b88r78OSLP9wnPnDMsswFKhJ', result_files=['file-lxucRnZDvrP7NBhFt39M9qkK'], seed=998221641, status='succeeded', trained_tokens=15465, training_file='file-uyhpnihtjozF9JK0y34ZQUK1', validation_file='file-Z1RarPDddirYwvf3B5v7ffCC', estimated_finish=None, integrations=[], user_provided_suffix='ner 4')

In [ ]:
model_name= client.fine_tuning.jobs.retrieve(job_id).fine_tuned_model # model ismini çekiyoruz.
model_name

'ft:gpt-4o-mini-2024-07-18:personal:ner-4:9uxSYzGm'

In [ ]:
system

"Extract the labels delimited by XML tags from text delimited by XML tags and return them in dict type.\n<labels>'Person': <person names and surnames>, 'Time': <word or words indicating time like this week, last month, summer, next month etc.>', 'Currency':<all currencies like USD, EURO, etc.>, 'Place':<settlement like country, city, district etc.> </labels>"

In [ ]:
user="My grandmother was a talented seamstress, and she taught me how to sew when I was young, which sparked a lifelong passion for fashion and design."
completion = client.chat.completions.create(
  model=model_name,
  messages=[
    {"role": "system", "content": system},
    {"role": "user", "content": '<text>'+user+'</text>' }
  ],
  temperature=0.0,
  top_p=1.0
)

print(completion.choices[0].message.content)

{'Person': 'null', 'Time': 'null', 'Currency': 'null', 'Place': 'null'}


In [ ]:
user="While I was on summer vacation, Joseph earned 100 dollars and 100 euros in Turkey and he returned to Germany"
completion = client.chat.completions.create(
  model=model_name,
  messages=[
    {"role": "system", "content": system},
    {"role": "user", "content": '<text>'+user+'</text>'}
  ],
  temperature=1.0,
  top_p=1.0
)

print(completion.choices[0].message.content)

{'Person': 'Joseph', 'Time': 'summer vacation', 'Currency': ['dollars', 'euros'], 'Place': ['Turkey', 'Germany']}


In [ ]:
user="Johnson Walker and Maria got married last month. They bought a house in London for 100,000 sterling pounds."
completion = client.chat.completions.create(
  model=model_name,
  messages=[
    {"role": "system", "content": system},
    {"role": "user", "content": '<text>'+user+'</text>'}
  ],
  temperature=0.0,
  top_p=1.0
)

print(completion.choices[0].message.content)

{'Person': ['Johnson Walker', 'Maria'], 'Time': 'last month', 'Currency': 'sterling pounds', 'Place': 'London'}


## Checkpoints

In [ ]:
events=client.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id, limit=100)

for event in events:
  if "full validation loss" in event.message:
    print(event.message)

Step 100/100: training loss=0.00, validation loss=0.00, full validation loss=0.00
Step 80/100: training loss=0.00, validation loss=0.00, full validation loss=0.00
Step 60/100: training loss=0.00, validation loss=0.00, full validation loss=0.00
Step 40/100: training loss=0.00, validation loss=0.00, full validation loss=0.00
Step 20/100: training loss=0.17, validation loss=0.03, full validation loss=0.01


In [ ]:
client.fine_tuning.jobs.checkpoints.list(fine_tuning_job_id=job_id).data

[FineTuningJobCheckpoint(id='ftckpt_Riivjv5LZBwgCh1UgsedzofR', created_at=1723361917, fine_tuned_model_checkpoint='ft:gpt-4o-mini-2024-07-18:personal:ner-4:9uxSYzGm', fine_tuning_job_id='ftjob-aCzu49O9poTR2wJBdRNUZjsg', metrics=Metrics(full_valid_loss=None, full_valid_mean_token_accuracy=None, step=100.0, train_loss=3.007742179761408e-06, train_mean_token_accuracy=1.0, valid_loss=3.6027696397569444e-06, valid_mean_token_accuracy=1.0), object='fine_tuning.job.checkpoint', step_number=100),
 FineTuningJobCheckpoint(id='ftckpt_sgzzaz5lZxOPaybYZNhHmfEs', created_at=1723361842, fine_tuned_model_checkpoint='ft:gpt-4o-mini-2024-07-18:personal:ner-4:9uxSYMyw:ckpt-step-80', fine_tuning_job_id='ftjob-aCzu49O9poTR2wJBdRNUZjsg', metrics=Metrics(full_valid_loss=None, full_valid_mean_token_accuracy=None, step=80.0, train_loss=3.814697265625e-06, train_mean_token_accuracy=1.0, valid_loss=2.966986762152778e-06, valid_mean_token_accuracy=1.0), object='fine_tuning.job.checkpoint', step_number=80),
 Fine

In [ ]:
model_name_ckpt = client.fine_tuning.jobs.checkpoints.list(fine_tuning_job_id=job_id).data[2].fine_tuned_model_checkpoint
model_name_ckpt

'ft:gpt-4o-mini-2024-07-18:personal:ner-4:9uxSY9yn:ckpt-step-60'

In [ ]:
user="My grandmother was a talented seamstress, and she taught me how to sew when I was young, which sparked a lifelong passion for fashion and design."
completion = client.chat.completions.create(
  model=model_name_ckpt,
  messages=[
    {"role": "system", "content": system},
    {"role": "user", "content": '<text>'+user+'</text>' }
  ],
  temperature=0.0,
  top_p=1.0
)

print(completion.choices[0].message.content)

{'Person': 'null', 'Time': 'null', 'Currency': 'null', 'Place': 'null'}


In [ ]:
user="Johnson Walker and Maria got married last month. They bought a house in London for 100,000 sterling pounds."
completion = client.chat.completions.create(
  model=model_name_ckpt,
  messages=[
    {"role": "system", "content": system},
    {"role": "user", "content": '<text>'+user+'</text>'}
  ],
  temperature=0.0,
  top_p=1.0
)

print(completion.choices[0].message.content)

{'Person': ['Johnson Walker', 'Maria'], 'Time': 'last month', 'Currency': 'sterling pounds', 'Place': 'London'}
